## I start with loading necessary libraries and indicating  standardized paths.

In [5]:
import pandas as pd
import os
import sys
import json
from pathlib import Path

In [8]:
# Define the root path of the project (two levels up from the notebooks folder)
ROOT = Path().resolve().parent

# Define paths for raw data files within the project
RAW_DATA_PATH = ROOT / 'data' / 'raw'
TRANSACTIONS_DATA_PATH = RAW_DATA_PATH / 'transactions_data.csv'
CLIENT_DATA_PATH = RAW_DATA_PATH / 'users_data.csv'
CARDS_DATA_PATH = RAW_DATA_PATH / 'cards_data.csv'
MCC_CODES_PATH = RAW_DATA_PATH / 'mcc_codes.json'

# Define paths for processed data or outputs (if needed)
PREDICTIONS_PATH = ROOT / 'predictions'
REPORTS_PATH = ROOT / 'reports'

In [10]:
# Load datasets
# Each dataset is loaded from a dynamically generated path
cards_data = pd.read_csv(CARDS_DATA_PATH)
users_data = pd.read_csv(CLIENT_DATA_PATH)
transactions_data = pd.read_csv(TRANSACTIONS_DATA_PATH)
mcc_codes_data = pd.read_json(MCC_CODES_PATH, orient='index')

In [11]:
# Display the first few rows of each dataset to understand its structure
cards_data.head()  # Shows card-related information

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
1,2731,825,Visa,Debit,4956965974959986,12/2020,393,YES,2,$21968,04/2014,2014,No
2,3701,825,Visa,Debit,4582313478255491,02/2024,719,YES,2,$46414,07/2003,2004,No
3,42,825,Visa,Credit,4879494103069057,08/2024,693,NO,1,$12400,01/2003,2012,No
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,03/2009,75,YES,1,$28,09/2008,2009,No


In [12]:
users_data.head()  # Shows user demographic and financial information

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,$29278,$59696,$127613,787,5
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,$37891,$77254,$191349,701,5
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,$22681,$33483,$196,698,5
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,$163145,$249925,$202328,722,4
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,$53797,$109687,$183855,675,1


In [13]:
transactions_data.head()  # Shows transaction details

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,NaN
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,NaN
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,NaN


In [14]:
mcc_codes_data.head()  # Shows merchant category codes

,0
5812,Eating Places and Restaurants
5541,Service Stations
7996,"Amusement Parks, Carnivals, Circuses"
5411,"Grocery Stores, Supermarkets"
4784,Tolls and Bridge Fees


In [15]:
# Preprocess mcc_codes_data
# Reset index to flatten the DataFrame and rename columns for clarity
mcc_codes_data.reset_index(inplace=True)
mcc_codes_data.columns = ["mcc_code", "category"]
mcc_codes_data = mcc_codes_data.rename(columns={mcc_codes_data.columns[0]: 'mcc'})

In [16]:
# Check basic information for the users_data dataset
# This will help identify any missing values and data types for each column
users_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 2000 non-null   int64  
 1   current_age        2000 non-null   int64  
 2   retirement_age     2000 non-null   int64  
 3   birth_year         2000 non-null   int64  
 4   birth_month        2000 non-null   int64  
 5   gender             2000 non-null   object 
 6   address            2000 non-null   object 
 7   latitude           2000 non-null   float64
 8   longitude          2000 non-null   float64
 9   per_capita_income  2000 non-null   object 
 10  yearly_income      2000 non-null   object 
 11  total_debt         2000 non-null   object 
 12  credit_score       2000 non-null   int64  
 13  num_credit_cards   2000 non-null   int64  
dtypes: float64(2), int64(7), object(5)
memory usage: 218.9+ KB


In [17]:
# Convert financial columns to integers by removing dollar signs
# This step is necessary for consistent data processing and analysis
users_data['per_capita_income'] = users_data['per_capita_income'].replace('[\$,]', '', regex=True).astype(int)
users_data['yearly_income'] = users_data['yearly_income'].replace('[\$,]', '', regex=True).astype(int)
users_data['total_debt'] = users_data['total_debt'].replace('[\$,]', '', regex=True).astype(int)

# Rename columns for clarity, indicating that these values are in dollars
users_data = users_data.rename(columns={
    "per_capita_income": "per_capita_income_$",
    "yearly_income": "yearly_income_$",
    "total_debt": "total_debt_$"
})

In [18]:
# Rename columns for clarity, indicating that these values are in dollars
users_data = users_data.rename(columns={
    "per_capita_income": "per_capita_income_$",
    "yearly_income": "yearly_income_$",
    "total_debt": "total_debt_$"
})

In [19]:
# Check for missing values in the users_data dataset to decide on handling strategy
users_data.isna().sum()

id                     0
current_age            0
retirement_age         0
birth_year             0
birth_month            0
gender                 0
address                0
latitude               0
longitude              0
per_capita_income_$    0
yearly_income_$        0
total_debt_$           0
credit_score           0
num_credit_cards       0
dtype: int64

In [20]:
# Perform similar preprocessing on the cards_data dataset
# Convert credit limits to integers by removing dollar signs
cards_data['credit_limit'] = cards_data['credit_limit'].replace('[\$,]', '', regex=True).astype(int)

# Convert account opening and expiration dates to datetime format
# This allows for easy date-based calculations and sorting later
cards_data['acct_open_date'] = pd.to_datetime(cards_data['acct_open_date'], format='%m/%Y', errors='coerce')
cards_data['expires'] = pd.to_datetime(cards_data['expires'], format='%m/%Y', errors='coerce')

# Rename columns for clarity in the cards_data dataset
cards_data = cards_data.rename(columns={
    "credit_limit": "credit_limit_$"
})

In [21]:
# Convert categorical columns to boolean values for easier analysis
# This step simplifies working with categorical features in later stages of modeling
cards_data['has_chip'] = cards_data['has_chip'].replace({'YES': True, 'NO': False})
cards_data['card_on_dark_web'] = cards_data['card_on_dark_web'].replace({'Yes': True, 'No': False})

# Drop columns with single unique values, as they do not provide useful information for analysis
cards_data = cards_data.drop(columns=['card_on_dark_web'])

/var/folders/xz/j9lwkzgn22xb5_ljk2ss2gpm0000gn/T/ipykernel_34550/726606257.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cards_data['has_chip'] = cards_data['has_chip'].replace({'YES': True, 'NO': False})
/var/folders/xz/j9lwkzgn22xb5_ljk2ss2gpm0000gn/T/ipykernel_34550/726606257.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cards_data['card_on_dark_web'] = cards_data['card_on_dark_web'].replace({'Yes': True, 'No': False})


In [22]:
# Now to the transactions data
transactions_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13305915 entries, 0 to 13305914
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   id              int64  
 1   date            object 
 2   client_id       int64  
 3   card_id         int64  
 4   amount          object 
 5   use_chip        object 
 6   merchant_id     int64  
 7   merchant_city   object 
 8   merchant_state  object 
 9   zip             float64
 10  mcc             int64  
 11  errors          object 
dtypes: float64(1), int64(5), object(6)
memory usage: 1.2+ GB


In [23]:
transactions_data.head()

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,NaN
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,NaN
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,NaN


In [24]:
# Convert date columns in transactions_data to datetime format
# This step enables date-based operations and analysis
transactions_data['date'] = pd.to_datetime(transactions_data['date'])

# Convert transaction amount to float after removing any currency symbols
transactions_data['amount'] = transactions_data['amount'].replace('[\$,]', '', regex=True).astype(float)

# Clean up zip codes and merchant states by filling in missing values
transactions_data['zip'] = transactions_data['zip'].astype(int, errors='ignore').fillna("Unknown")
transactions_data["merchant_state"] = transactions_data["merchant_state"].fillna("Unknown")
transactions_data['errors'] = transactions_data['errors'].fillna("No")
transactions_data['time'] = pd.to_datetime(transactions_data['date']).dt.time

## After the initial cleaning process of each dataset, I proceeded to merge mcc codes with transactions.

In [25]:
transactions_data = transactions_data.merge(mcc_codes_data, on="mcc", how="left")

In [26]:
transactions_data.head()

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,time,category
0,7475327,2010-01-01 00:01:00,1556,2972,-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,No,00:01:00,Miscellaneous Food Stores
1,7475328,2010-01-01 00:02:00,561,4575,14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,No,00:02:00,Department Stores
2,7475329,2010-01-01 00:02:00,1129,102,80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,No,00:02:00,Money Transfer
3,7475331,2010-01-01 00:05:00,430,2860,200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,No,00:05:00,Money Transfer
4,7475332,2010-01-01 00:06:00,848,3915,46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,No,00:06:00,Drinking Places (Alcoholic Beverages)


In [27]:
# The mcc codes are no longer needed, so we can drop the column
transactions_data = transactions_data.drop(columns=['mcc'])

In [28]:
transactions_data.head()

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,errors,time,category
0,7475327,2010-01-01 00:01:00,1556,2972,-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,No,00:01:00,Miscellaneous Food Stores
1,7475328,2010-01-01 00:02:00,561,4575,14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,No,00:02:00,Department Stores
2,7475329,2010-01-01 00:02:00,1129,102,80.00,Swipe Transaction,27092,Vista,CA,92084.0,No,00:02:00,Money Transfer
3,7475331,2010-01-01 00:05:00,430,2860,200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,No,00:05:00,Money Transfer
4,7475332,2010-01-01 00:06:00,848,3915,46.41,Swipe Transaction,13051,Harwood,MD,20776.0,No,00:06:00,Drinking Places (Alcoholic Beverages)


## Now I can merge clients_data with cards_data as both have client_id column.

In [29]:
client_card_data = pd.merge(users_data, cards_data, left_on="id", right_on="client_id", how="left")
client_card_data.head()

,id_x,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income_$,...,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit_$,acct_open_date,year_pin_last_changed
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,29278,...,Visa,Debit,4344676511950444,2022-12-01,623,True,2,24295,2002-09-01,2008
1,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,29278,...,Visa,Debit,4956965974959986,2020-12-01,393,True,2,21968,2014-04-01,2014
2,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,29278,...,Visa,Debit,4582313478255491,2024-02-01,719,True,2,46414,2003-07-01,2004
3,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,29278,...,Visa,Credit,4879494103069057,2024-08-01,693,False,1,12400,2003-01-01,2012
4,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,29278,...,Mastercard,Debit (Prepaid),5722874738736011,2009-03-01,75,True,1,28,2008-09-01,2009


In [30]:
client_card_data.columns

Index(['id_x', 'current_age', 'retirement_age', 'birth_year', 'birth_month',
       'gender', 'address', 'latitude', 'longitude', 'per_capita_income_$',
       'yearly_income_$', 'total_debt_$', 'credit_score', 'num_credit_cards',
       'id_y', 'client_id', 'card_brand', 'card_type', 'card_number',
       'expires', 'cvv', 'has_chip', 'num_cards_issued', 'credit_limit_$',
       'acct_open_date', 'year_pin_last_changed'],
      dtype='object')

In [31]:
client_card_data = client_card_data.rename(columns={"id_x": "user_id", "id_y": "card_id"})

In [32]:
client_card_data

,user_id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income_$,...,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit_$,acct_open_date,year_pin_last_changed
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,29278,...,Visa,Debit,4344676511950444,2022-12-01,623,True,2,24295,2002-09-01,2008
1,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,29278,...,Visa,Debit,4956965974959986,2020-12-01,393,True,2,21968,2014-04-01,2014
2,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,29278,...,Visa,Debit,4582313478255491,2024-02-01,719,True,2,46414,2003-07-01,2004
3,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,29278,...,Visa,Credit,4879494103069057,2024-08-01,693,False,1,12400,2003-01-01,2012
4,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,29278,...,Mastercard,Debit (Prepaid),5722874738736011,2009-03-01,75,True,1,28,2008-09-01,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6141,185,47,67,1973,1,Female,276 Fifth Boulevard,40.66,-74.19,15175,...,Amex,Credit,300609782832003,2024-01-01,663,True,1,6900,2000-11-01,2013
6142,185,47,67,1973,1,Female,276 Fifth Boulevard,40.66,-74.19,15175,...,Visa,Credit,4718517475996018,2021-01-01,492,True,2,5700,2012-04-01,2012
6143,1007,66,60,1954,2,Male,259 Valley Boulevard,40.24,-76.92,25336,...,Mastercard,Credit,5929512204765914,2020-08-01,237,False,2,9200,2012-02-01,2012
6144,1110,21,60,1998,11,Female,472 Ocean View Street,42.86,-71.48,32325,...,Mastercard,Debit,5589768928167462,2020-01-01,630,True,1,28074,2020-01-01,2020


Time to merge transaction with client and card data.

In [33]:
full_data = pd.merge(transactions_data, client_card_data, on="card_id", how="left")

In [34]:
full_data.head()

,id,date,client_id_x,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,...,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit_$,acct_open_date,year_pin_last_changed
0,7475327,2010-01-01 00:01:00,1556,2972,-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,...,Mastercard,Debit (Prepaid),5497590243197280,2022-07-01,306,True,2,55,2008-05-01,2008
1,7475328,2010-01-01 00:02:00,561,4575,14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,...,Mastercard,Credit,5175842699412235,2024-12-01,438,True,1,9100,2005-09-01,2015
2,7475329,2010-01-01 00:02:00,1129,102,80.00,Swipe Transaction,27092,Vista,CA,92084.0,...,Mastercard,Debit,5874992802287595,2020-05-01,256,True,1,14802,2006-01-01,2008
3,7475331,2010-01-01 00:05:00,430,2860,200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,...,Mastercard,Debit,5346827663529174,2024-10-01,54,False,2,37634,2004-05-01,2006
4,7475332,2010-01-01 00:06:00,848,3915,46.41,Swipe Transaction,13051,Harwood,MD,20776.0,...,Visa,Debit,4354185735186651,2020-01-01,120,True,1,19113,2009-07-01,2014


In [35]:
full_data.columns

Index(['id', 'date', 'client_id_x', 'card_id', 'amount', 'use_chip',
       'merchant_id', 'merchant_city', 'merchant_state', 'zip', 'errors',
       'time', 'category', 'user_id', 'current_age', 'retirement_age',
       'birth_year', 'birth_month', 'gender', 'address', 'latitude',
       'longitude', 'per_capita_income_$', 'yearly_income_$', 'total_debt_$',
       'credit_score', 'num_credit_cards', 'client_id_y', 'card_brand',
       'card_type', 'card_number', 'expires', 'cvv', 'has_chip',
       'num_cards_issued', 'credit_limit_$', 'acct_open_date',
       'year_pin_last_changed'],
      dtype='object')

In [36]:
full_data = full_data.rename(columns={"client_id_x": "client_id"})
full_data = full_data.drop(columns=["client_id_y"])

In [37]:
full_data.isna().sum()

id                       0
date                     0
client_id                0
card_id                  0
amount                   0
use_chip                 0
merchant_id              0
merchant_city            0
merchant_state           0
zip                      0
errors                   0
time                     0
category                 0
user_id                  0
current_age              0
retirement_age           0
birth_year               0
birth_month              0
gender                   0
address                  0
latitude                 0
longitude                0
per_capita_income_$      0
yearly_income_$          0
total_debt_$             0
credit_score             0
num_credit_cards         0
card_brand               0
card_type                0
card_number              0
expires                  0
cvv                      0
has_chip                 0
num_cards_issued         0
credit_limit_$           0
acct_open_date           0
year_pin_last_changed    0
d

In [38]:
# Check for duplicated entries in each dataset
print("Duplicated rows in full_data:", full_data.duplicated().sum())

Duplicated rows in full_data: 0


Now all the data is in the same dataset, is cleaned, has non NaNs or duplicates. 

## Task 1, Query 1.

### query_1: The card_id with the latest expiry date and the lowest credit limit amount.

In [39]:
# Step 1: Find the latest expiry date across all cards
latest_expiry_date = full_data['expires'].max()
# Step 2: Filter data for cards with this expiry date
latest_expiry_date_df = full_data[full_data['expires'] == latest_expiry_date]
print("Latest expiry date details:", latest_expiry_date_df)

# Step 3: Find the lowest credit limit among these cards
lowest_credit_limit = latest_expiry_date_df['credit_limit_$'].min()
print("Lowest credit limit with latest expiry:", lowest_credit_limit)

# Alternatively, find the card_id with the latest expiry date and lowest credit limit in a single step
latest_card = full_data.sort_values(by=["expires", "credit_limit_$"], ascending=[False, True]).iloc[0]["card_id"]
query_1 = latest_card  # Store the result for saving

print("Card ID for query 1:", query_1)


Latest expiry date details:                 id                date  client_id  card_id  amount  \
1          7475328 2010-01-01 00:02:00        561     4575   14.57   
37         7475367 2010-01-01 01:06:00       1758     4686   87.09   
315        7475693 2010-01-01 06:37:00       1430     2813   56.00   
352        7475736 2010-01-01 06:48:00       1430     2813   58.60   
358        7475742 2010-01-01 06:50:00       1430     2813  -56.00   
...            ...                 ...        ...      ...     ...   
13305606  23761496 2019-10-31 20:48:00       1985     4085   10.74   
13305689  23761589 2019-10-31 21:24:00       1274     3200    1.52   
13305712  23761617 2019-10-31 21:32:00        220     2697   37.16   
13305816  23761752 2019-10-31 22:31:00       1996     4212   35.82   
13305830  23761768 2019-10-31 22:39:00        704     2945   20.67   

                    use_chip  merchant_id merchant_city merchant_state  \
1          Swipe Transaction        67570    Bettendorf  

### query_2: The client_id that will retire within a year that has the lowest credit score and highest debt.

In [40]:
# Filter clients who will retire within a year
retiring_clients = full_data[(full_data["retirement_age"] - full_data["current_age"] <= 1)] 
print("Number of clients retiring within a year:", retiring_clients["client_id"].nunique())

# Sort these clients by lowest credit score and highest debt, and select the first one
query_2 = retiring_clients.sort_values(by=["credit_score", "total_debt_$"], ascending=[True, False]).iloc[0]["client_id"]
print("Client ID for query 2:", query_2)

Number of clients retiring within a year: 266
Client ID for query 2: 1987


In [41]:
query_2 = retiring_clients.sort_values(by=["credit_score", "total_debt_$"], ascending=[True, False])[['client_id', 'credit_score', 'total_debt_$']].iloc[0]["client_id"]

### query_3: The transaction_id of an Online purchase on a 31st of December with the highest absolute amount (either earnings or expenses).

In [42]:
full_data['date'] = pd.to_datetime(full_data['date'])

In [43]:
# Find the transaction with the highest absolute amount on December 31st that was made online
max_idx = full_data[(full_data['date'].dt.month == 12) &
                    (full_data['date'].dt.day == 31) &
                    (full_data['use_chip'] == 'Online Transaction')]['amount'].abs().idxmax()

# Extract transaction ID for this transaction
online_purchase = full_data.loc[max_idx, 'id']
query_3 = online_purchase  # Store the result for saving

print('Transaction ID for query 3:', query_3)

Transaction ID for query 3: 10534178


### query_4: Which client over the age of 40 made the most transactions with a Visa card in February 2016? Please return the client_id, the card_id involved, and the total number of transactions.

In [44]:
## Check for unique values in the card_brand column
full_data['card_brand'].unique().tolist()

['Mastercard', 'Visa', 'Amex', 'Discover']

In [45]:
# Filter for clients over 40 who used a Visa card in February 2016
feb_visa_transactions = full_data[(full_data['current_age'] > 40) &
                                  (full_data['card_brand'] == 'Visa') &
                                  (full_data['date'].dt.year == 2016) &
                                  (full_data['date'].dt.month == 2)]

# Identify the client with the most transactions
top_client = feb_visa_transactions["client_id"].value_counts().idxmax()

# Get the corresponding card ID and total transaction count for this client
top_card_id = feb_visa_transactions[feb_visa_transactions['client_id'] == top_client]['card_id'].iloc[0]
total_transactions = feb_visa_transactions['client_id'].value_counts().max()

print('Client ID for query 4:', top_client)
print('Card ID for query 4:', top_card_id)
print('Total transactions for query 4:', total_transactions)

Client ID for query 4: 1963
Card ID for query 4: 6058
Total transactions for query 4: 276


### Prepare the queries to be saved as JSON for automatic validation.

In [46]:
results = {
    "target": {
        "query_1": {
            "card_id": int(query_1)
        },
        "query_2": {
            "client_id": int(query_2)  
        },
        "query_3": {
            "transaction_id": int(query_3)
        },
        "query_4": {
            "client_id": int(top_client),
            "card_id": int(top_card_id),
            "number_transactions": int(total_transactions)
        }
    }
}

In [47]:
# Define the path for saving the JSON output
OUTPUT_PATH = os.path.join(ROOT, 'predictions', 'predictions_1.json')

# Save the results to a JSON file
import json
with open(OUTPUT_PATH, 'w') as json_file:
    json.dump(results, json_file, indent=4)
print("Results saved to:", OUTPUT_PATH)

Results saved to: /Users/nataliiapopkova/Downloads/portfolio/hackaton/hackathon-caixabank-data-ai-report/predictions/predictions_1.json
